In [1]:
import pandas as pd
from collections import defaultdict
import warnings
import visualize
import neat
import multiprocessing
import os
import matplotlib.pyplot as plt

import numpy as np

In [2]:
necoc = 5
generations = int(3000/necoc)
generations

600

In [3]:
def softmax(x, axis=1):
    # 计算每行的最大值
    row_max = x.max(axis=axis)
 
    # 每行元素都需要减去对应的最大值，否则求exp(x)会溢出，导致inf情况
    row_max=row_max.reshape(-1, 1)
    x = x - row_max
 
    # 计算e的指数次幂
    x_exp = np.exp(x)
    x_sum = np.sum(x_exp, axis=axis, keepdims=True)
    s = x_exp / x_sum
    return s

In [4]:
df = pd.read_csv('letter-recognition.data',header=None)

train_df = df[:1600]
test_df = df[16000:16000+400]

X = train_df.ix[:,1:16].to_numpy() 
y = np.array([ord(l)-ord('A') for l in train_df.ix[:,0].to_numpy()])

In [5]:
def get_winner(class1):
    
    numbers = [*range(26)] # number 2 > number 1
    number_of_classification = len(numbers)
    #number_of_sampling = 100

    list_y = y.tolist()
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

        
    class2 = []

    class1_indexes = []
    class2_indexes = []

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)

    print("number of class1:{}".format(len(class1_label)))
    print("number of class2:{}".format(len(class2_label)))

    samplesize = 1600
    x_inputs =[tuple(c) for c in X[class1_indexes].tolist()] + [tuple(c) for c in X[class2_indexes].tolist()]
    x_outputs = [tuple([c]) for c in class1_label + class2_label]
    
    
    def eval_genomes(genomes, config):
        for genome_id, genome in genomes:
            net = neat.nn.FeedForwardNetwork.create(genome, config)

            outputs = []
            for xi in x_inputs:
                output = net.activate(xi)
                outputs.append(output)


            px_outputs = softmax(np.array(outputs).reshape(samplesize, 2), axis=1)
            # the index of maximum in each line
            pred_outputs = np.argmax(px_outputs, axis = 1)
            real_outputs = np.array(x_outputs).reshape(samplesize,)

            acc = np.sum(pred_outputs == real_outputs)/samplesize

            genome.fitness = acc

    def run(config_file):
        # Load configuration.
        config = neat.Config(
            neat.DefaultGenome,
            neat.DefaultReproduction,
            neat.DefaultSpeciesSet,
            neat.DefaultStagnation,
            config_file,
        )

        # Create the population, which is the top-level object for a NEAT run.
        p = neat.Population(config)

        # add a stdout reporter to show progress in the terminal
        reporter = neat.StdOutReporter(False)
        p.add_reporter(reporter)
        stats = neat.StatisticsReporter()
        p.add_reporter(stats)
        #checkpointer = neat.Checkpointer(100)
        #p.add_reporter(checkpointer)
        # Run for up to 300 generations.
        winner = p.run(eval_genomes, generations)

        return [stats, winner]
    
    local_dir = os.getcwd()
    config_path = os.path.join(local_dir, "config-feedforward-letter2")
    [stats, winner] = run(config_path)
    
    return winner

In [11]:
import numpy as np
import random

number_codes = []
for i in range(2**(26-1), 2**26-1):
    if (i % 1000000) == 0:
            print(i)
    for j in bin(i).split('b')[1]:
        number_codes.append(int(j))
        


number_codes1 = np.array(number_codes).reshape(2**(26 - 1) - 1,26).T     

34000000
35000000
36000000
37000000
38000000
39000000
40000000
41000000
42000000
43000000
44000000
45000000
46000000
47000000
48000000
49000000
50000000
51000000
52000000
53000000
54000000
55000000
56000000
57000000
58000000
59000000
60000000
61000000
62000000
63000000
64000000
65000000
66000000
67000000


In [18]:
while True:
    slice = random.sample(range(2**(26 - 1) - 1), necoc)

    number_codes = number_codes1[:,slice].tolist()

    classes = []
    for i in range(necoc):
        code_list = np.array(number_codes).T[i]
        classes.append(np.where(code_list==1)[0].tolist())

    matrix_errors = []

    for i in range(25):
        for j in range(i+1,26):
            matrix_errors.append(necoc - np.sum(np.array(number_codes[i]) == np.array(number_codes[j])))
    min_error = np.min(matrix_errors)
    if min_error != 0:
        print(min_error)
        break

1


In [19]:
winner_list = []

for i,class1 in enumerate(classes):
    print("running:{}/5".format(i))
    winner = get_winner(class1)
    winner_list.append(winner)

running:0/5
number of class1:845
number of class2:755

 ****** Running generation 0 ****** 

Population's average fitness: 0.49671 stdev: 0.02754
Best fitness: 0.57063 - size: (2, 3) - species 3 - id 138
Average adjusted fitness: 0.077
Mean genetic distance 2.644, standard deviation 0.499
Population of 200 members in 12 species
Total extinctions: 0
Generation time: 4.709 sec

 ****** Running generation 1 ****** 

Population's average fitness: 0.51462 stdev: 0.03079
Best fitness: 0.62813 - size: (3, 4) - species 1 - id 286
Average adjusted fitness: 0.106
Mean genetic distance 2.612, standard deviation 0.500
Population of 201 members in 13 species
Total extinctions: 0
Generation time: 2.936 sec (3.822 average)

 ****** Running generation 2 ****** 

Population's average fitness: 0.51596 stdev: 0.03507
Best fitness: 0.63687 - size: (3, 4) - species 5 - id 547
Average adjusted fitness: 0.081
Mean genetic distance 2.677, standard deviation 0.551
Population of 200 members in 16 species
Total 

Population's average fitness: 0.54280 stdev: 0.05053
Best fitness: 0.64125 - size: (5, 10) - species 4 - id 2912
Average adjusted fitness: 0.100
Mean genetic distance 3.207, standard deviation 0.575
Population of 198 members in 25 species
Total extinctions: 0
Generation time: 4.100 sec (4.125 average)

 ****** Running generation 25 ****** 

Population's average fitness: 0.54684 stdev: 0.04680
Best fitness: 0.64125 - size: (5, 10) - species 4 - id 2912
Average adjusted fitness: 0.089
Mean genetic distance 3.213, standard deviation 0.579
Population of 205 members in 24 species
Total extinctions: 0
Generation time: 4.282 sec (4.154 average)

 ****** Running generation 26 ****** 

Population's average fitness: 0.54288 stdev: 0.04964
Best fitness: 0.64125 - size: (5, 10) - species 4 - id 2912
Average adjusted fitness: 0.141
Mean genetic distance 3.220, standard deviation 0.562
Population of 198 members in 25 species
Total extinctions: 0
Generation time: 4.544 sec (4.202 average)

 ****** Ru

Population's average fitness: 0.54479 stdev: 0.05178
Best fitness: 0.65750 - size: (6, 15) - species 4 - id 6904
Average adjusted fitness: 0.112
Mean genetic distance 3.172, standard deviation 0.628
Population of 195 members in 15 species
Total extinctions: 0
Generation time: 4.720 sec (4.689 average)

 ****** Running generation 49 ****** 

Population's average fitness: 0.54450 stdev: 0.05397
Best fitness: 0.65750 - size: (6, 15) - species 4 - id 6904
Average adjusted fitness: 0.105
Mean genetic distance 3.188, standard deviation 0.672
Population of 201 members in 13 species
Total extinctions: 0
Generation time: 4.600 sec (4.680 average)

 ****** Running generation 50 ****** 

Population's average fitness: 0.53634 stdev: 0.05363
Best fitness: 0.65750 - size: (6, 15) - species 4 - id 6904
Average adjusted fitness: 0.086
Mean genetic distance 3.170, standard deviation 0.680
Population of 200 members in 14 species
Total extinctions: 0
Generation time: 4.691 sec (4.656 average)

 ****** Ru

Generation time: 5.047 sec (5.011 average)

 ****** Running generation 73 ****** 

Population's average fitness: 0.54807 stdev: 0.05515
Best fitness: 0.66500 - size: (7, 19) - species 4 - id 9414
Average adjusted fitness: 0.096
Mean genetic distance 3.311, standard deviation 0.557
Population of 195 members in 20 species
Total extinctions: 0
Generation time: 4.845 sec (4.997 average)

 ****** Running generation 74 ****** 

Population's average fitness: 0.55051 stdev: 0.05463
Best fitness: 0.66500 - size: (7, 19) - species 4 - id 9414
Average adjusted fitness: 0.093
Mean genetic distance 3.297, standard deviation 0.588
Population of 203 members in 18 species
Total extinctions: 0
Generation time: 4.704 sec (4.950 average)

 ****** Running generation 75 ****** 

Population's average fitness: 0.54949 stdev: 0.05462
Best fitness: 0.66500 - size: (7, 19) - species 4 - id 9414
Average adjusted fitness: 0.135
Mean genetic distance 3.311, standard deviation 0.580
Population of 194 members in 19 

Mean genetic distance 3.234, standard deviation 0.596
Population of 202 members in 19 species
Total extinctions: 0
Generation time: 5.612 sec (5.364 average)

 ****** Running generation 98 ****** 

Population's average fitness: 0.53894 stdev: 0.05126
Best fitness: 0.66500 - size: (7, 16) - species 4 - id 13856
Average adjusted fitness: 0.104
Mean genetic distance 3.230, standard deviation 0.583
Population of 199 members in 19 species
Total extinctions: 0
Generation time: 5.950 sec (5.465 average)

 ****** Running generation 99 ****** 

Population's average fitness: 0.54232 stdev: 0.05433
Best fitness: 0.66500 - size: (7, 16) - species 4 - id 13856
Average adjusted fitness: 0.116
Mean genetic distance 3.261, standard deviation 0.589
Population of 204 members in 17 species
Total extinctions: 0
Generation time: 5.935 sec (5.530 average)

 ****** Running generation 100 ****** 

Population's average fitness: 0.54468 stdev: 0.05258
Best fitness: 0.66500 - size: (7, 16) - species 4 - id 13856

...

Mean genetic distance 2.892, standard deviation 0.574
Population of 200 members in 9 species
Total extinctions: 0
Generation time: 15.810 sec (15.724 average)

 ****** Running generation 499 ****** 

Population's average fitness: 0.55410 stdev: 0.05709
Best fitness: 0.70312 - size: (25, 57) - species 202 - id 72331
Average adjusted fitness: 0.103
Mean genetic distance 2.897, standard deviation 0.554
Population of 200 members in 9 species
Total extinctions: 0
Generation time: 15.677 sec (15.709 average)

 ****** Running generation 500 ****** 

Population's average fitness: 0.55185 stdev: 0.05566
Best fitness: 0.70312 - size: (25, 57) - species 202 - id 72331
Average adjusted fitness: 0.078
Mean genetic distance 2.894, standard deviation 0.547
Population of 199 members in 9 species
Total extinctions: 0
Generation time: 15.884 sec (15.704 average)

 ****** Running generation 501 ****** 

Population's average fitness: 0.55557 stdev: 0.05235
Best fitness: 0.70312 - size: (25, 57) - species 

Population's average fitness: 0.54985 stdev: 0.05808
Best fitness: 0.70312 - size: (25, 57) - species 202 - id 72331
Average adjusted fitness: 0.077
Mean genetic distance 2.950, standard deviation 0.508
Population of 195 members in 16 species
Total extinctions: 0
Generation time: 16.710 sec (16.424 average)

 ****** Running generation 523 ****** 

Population's average fitness: 0.55354 stdev: 0.05934
Best fitness: 0.70312 - size: (25, 57) - species 202 - id 72331
Average adjusted fitness: 0.115
Mean genetic distance 2.945, standard deviation 0.535
Population of 198 members in 17 species
Total extinctions: 0
Generation time: 14.938 sec (16.346 average)

 ****** Running generation 524 ****** 

Population's average fitness: 0.55582 stdev: 0.05734
Best fitness: 0.70312 - size: (25, 57) - species 202 - id 72331
Average adjusted fitness: 0.089
Mean genetic distance 2.950, standard deviation 0.532
Population of 202 members in 17 species
Total extinctions: 0
Generation time: 15.579 sec (16.298 

Mean genetic distance 2.938, standard deviation 0.564
Population of 198 members in 12 species
Total extinctions: 0
Generation time: 17.458 sec (17.576 average)

 ****** Running generation 546 ****** 

Population's average fitness: 0.55346 stdev: 0.05625
Best fitness: 0.70312 - size: (25, 57) - species 202 - id 72331
Average adjusted fitness: 0.082
Mean genetic distance 2.927, standard deviation 0.570
Population of 197 members in 12 species
Total extinctions: 0
Generation time: 16.622 sec (17.499 average)

 ****** Running generation 547 ****** 

Population's average fitness: 0.55424 stdev: 0.05530
Best fitness: 0.70312 - size: (25, 57) - species 202 - id 72331
Average adjusted fitness: 0.088
Mean genetic distance 2.924, standard deviation 0.572
Population of 200 members in 12 species
Total extinctions: 0
Generation time: 17.119 sec (17.435 average)

 ****** Running generation 548 ****** 

Population's average fitness: 0.55037 stdev: 0.05731
Best fitness: 0.70312 - size: (25, 57) - speci

Population's average fitness: 0.55592 stdev: 0.05798
Best fitness: 0.70875 - size: (25, 56) - species 228 - id 95593
Average adjusted fitness: 0.100
Mean genetic distance 2.975, standard deviation 0.559
Population of 197 members in 15 species
Total extinctions: 0
Generation time: 20.042 sec (16.698 average)

 ****** Running generation 570 ****** 

Population's average fitness: 0.55849 stdev: 0.05854
Best fitness: 0.70875 - size: (25, 56) - species 228 - id 95593
Average adjusted fitness: 0.096
Mean genetic distance 2.985, standard deviation 0.563
Population of 197 members in 15 species
Total extinctions: 0
Generation time: 14.888 sec (16.518 average)

 ****** Running generation 571 ****** 

Population's average fitness: 0.55643 stdev: 0.05849
Best fitness: 0.70875 - size: (25, 56) - species 228 - id 95593
Average adjusted fitness: 0.095
Mean genetic distance 2.973, standard deviation 0.570
Population of 197 members in 15 species
Total extinctions: 0
Generation time: 14.460 sec (16.344 

Mean genetic distance 2.847, standard deviation 0.508
Population of 199 members in 14 species
Total extinctions: 0
Generation time: 13.992 sec (13.851 average)

 ****** Running generation 593 ****** 

Population's average fitness: 0.55175 stdev: 0.06198
Best fitness: 0.70875 - size: (25, 56) - species 228 - id 95593
Average adjusted fitness: 0.080
Mean genetic distance 2.820, standard deviation 0.519
Population of 200 members in 14 species
Total extinctions: 0
Generation time: 13.934 sec (13.806 average)

 ****** Running generation 594 ****** 

Population's average fitness: 0.54801 stdev: 0.06226
Best fitness: 0.70875 - size: (25, 56) - species 228 - id 95593
Average adjusted fitness: 0.096
Mean genetic distance 2.744, standard deviation 0.449
Population of 204 members in 13 species
Total extinctions: 0
Generation time: 13.939 sec (13.785 average)

 ****** Running generation 595 ****** 

Population's average fitness: 0.55438 stdev: 0.05698
Best fitness: 0.70875 - size: (25, 56) - speci

In [20]:
testX = test_df.ix[:,1:16].to_numpy() 
testy = np.array([ord(l)-ord('A') for l in test_df.ix[:,0].to_numpy()])

local_dir = os.getcwd()
config_path = os.path.join(local_dir, "config-feedforward-letter2")

config = neat.Config(
    neat.DefaultGenome,
    neat.DefaultReproduction,
    neat.DefaultSpeciesSet,
    neat.DefaultStagnation,
    config_path,
)

def get_pred_real(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = testX[j]
    test_y = testy[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

from random import randint
pred_value = []
error_list = []

for j in range(400):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_real(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(26):
        error.append(necoc - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )
list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
np.sum(list_P == testy)/400

0.175

In [21]:
def get_pred_train(i, j):
    winner_net = neat.nn.FeedForwardNetwork.create(winner_list[i], config)
    
    numbers = [*range(26)]
    class1 = classes[i]
    class2 = []
    class1_indexes = []
    class2_indexes = []
    
    test_X = X[j]
    test_y = y[j]

    list_y = [test_y]
    digits_indexes = []
    for digit in numbers:
        li = [i for i in range(len(list_y)) if list_y[i] == digit]
        digits_indexes.extend([li])

    for i in numbers:
        if i in class1:
            class1_indexes.extend(digits_indexes[i])
        else:
            class2_indexes.extend(digits_indexes[i])
            class2.append(i)

    class1_label = [1] * len(class1_indexes)
    class2_label = [0] * len(class2_indexes)


    testsamplesize = 1
    test_x_inputs =[tuple(test_X)]
    test_x_outputs = [tuple([c]) for c in class1_label + class2_label]

    outputs = []
    for xi in test_x_inputs:
        output = winner_net.activate(xi)
        outputs.append(output)

    px_outputs = softmax(np.array(outputs).reshape(testsamplesize, 2), axis=1)
    # the index of maximum in each line
    pred_outputs = np.argmax(px_outputs, axis = 1)
    real_outputs = np.array(test_x_outputs).reshape(testsamplesize,)

    
    return [pred_outputs, real_outputs]

pred_value = []
error_list = []

for j in range(1600):
    pred = []
    for i in range(necoc):
        [pred_outputs, real_outputs] = get_pred_train(i, j)
        #print(pred_outputs, real_outputs)
        pred.append(pred_outputs)
    #print(np.array(pred).T)
    
    error = []
    for i in range(26):
        error.append(26 - np.sum(number_codes[i] == np.array(pred).T))
    #print(error)
    
    pred_value.append(np.where(error==np.min(error)))
    error_list.append(np.min(error))
    #print(np.where(error==np.min(error)) )

list_P = []
for i in pred_value:
    if (len(i[0])) == 1:
        list_P.append(i[0][0])
    else:
        random_pick = randint(0, len(i[0])-1)
        list_P.append(i[0][random_pick])
print("Trainset acc:{}".format(np.sum(list_P == y)/1600))
winner_fitness = []
for winner in winner_list:
    #print(winner.fitness)
    winner_fitness.append(winner.fitness)
print("Avg Base Learner:{}".format(np.mean(winner_fitness)))

Trainset acc:0.166875
Avg Base Learner:0.6901249999999999


In [22]:
# program to check if there is exist a path between two vertices 
# of a graph 

from collections import defaultdict 

#This class represents a directed graph using adjacency list representation 
class Graph: 

	def __init__(self,vertices): 
		self.V= vertices #No. of vertices 
		self.graph = defaultdict(list) # default dictionary to store graph 

	# function to add an edge to graph 
	def addEdge(self,u,v): 
		self.graph[u].append(v) 
	
	# Use BFS to check path between s and d 
	def isReachable(self, s, d): 
		# Mark all the vertices as not visited 
		visited =[False]*(self.V) 

		# Create a queue for BFS 
		queue=[] 

		# Mark the source node as visited and enqueue it 
		queue.append(s) 
		visited[s] = True

		while queue: 

			#Dequeue a vertex from queue 
			n = queue.pop(0) 
			
			# If this adjacent node is the destination node, 
			# then return true 
			if n == d: 
				return True

			# Else, continue to do BFS 
			for i in self.graph[n]: 
				if visited[i] == False: 
					queue.append(i) 
					visited[i] = True
		# If BFS is complete without visited d 
		return False
    
    
def findAllPath(graph,start,end,path=[]):
    path = path +[start]
    if start == end:
        return [path]
 
    paths = [] #存储所有路径    
    for node in graph[start]:
        if node not in path:
            newpaths = findAllPath(graph,node,end,path) 
            for newpath in newpaths:
                paths.append(newpath)
    return paths

numbers = [*range(2)]
number_of_classification = len(numbers)

list_nodes_number = []
list_connection_number = []

for winner in winner_list:
    used_nodes = list(winner.nodes.keys())
    # create a dict for mapping 
    l1 = numbers + config.genome_config.input_keys + used_nodes[number_of_classification:]
    l2 = range(len(l1))
    dict_nodes = dict(zip(l1, l2))

    ### 将节点加入图
    g = Graph(len(l1))# inputs + outputs + used  
    # add connections in the graph
    for cg in winner.connections.values():
        if cg.enabled:
            g.addEdge(dict_nodes[cg.key[0]],dict_nodes[cg.key[1]])

    v = numbers
    list_connections = []
    for vi in v:
        for u in range(number_of_classification, len(config.genome_config.input_keys) + number_of_classification):
            list_connections.append(g.isReachable(u, vi))

    nodes_inputs = range(number_of_classification, number_of_classification+ 64)
    nodes_outputs = range(number_of_classification)
    nodes_mid = range(number_of_classification + 64, len(dict_nodes))

    # 找到所有输入和输出连接的路
    all_path = []
    for u in nodes_inputs:
        for v in nodes_outputs:
            path = findAllPath(g.graph, u, v)
            if path:
                all_path = all_path + path

    # 得到最长路，以及每个节点在哪个层级
    max_length = max([len(x) for x in all_path])
    nodes_tuples_list = []
    for path in all_path:
        #print(path)
        for node in path:
            nodes_tuples_list.append([node, path.index(node)])

    # 确定节点的层级（消除重复）       
    nodes_tuples_fixed_list = []
    for index in range(max_length):
        for nodes in nodes_tuples_list:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                if nodes not in nodes_tuples_fixed_list:
                    nodes_tuples_fixed_list.append(nodes)


    # 所有有连接的节点
    all_used_nodes = []
    for node in nodes_tuples_fixed_list:
        if node[0] not in all_used_nodes:
            all_used_nodes.append(node[0])

    # 确定节点的层级（最终确定）
    nodes_tuples_fixed_list_final = []

    for node in all_used_nodes:
        if node in nodes_inputs:
            nodes_tuples_fixed_list_final.append([node, 0])
        elif node in nodes_outputs:
            nodes_tuples_fixed_list_final.append([node, max_length - 1])
        else:
            list_node = []
            for nodes in nodes_tuples_fixed_list:
                if node == nodes[0]:
                    list_node.append(nodes)
            layer = 0
            for ele in list_node:
                # 删除重复项，取最大
                if ele[1] > layer:
                    layer = ele[1]
            nodes_tuples_fixed_list_final.append([node,layer])


    # 每个层级的节点合数
    length_of_layers = []
    number_of_nodes = 0
    for index in range(max_length):
        for nodes in nodes_tuples_fixed_list_final:
            node, node_index = nodes[0], nodes[1]
            if node_index == index: 
                number_of_nodes += 1
        length_of_layers.append(number_of_nodes)

    length_of_each_layer = []
    length_of_each_layer.append(length_of_layers[0])
    for i in range(1, max_length):
        length_of_each_layer.append(length_of_layers[i] - length_of_layers[i-1])

    # 输出每个层级的节点个数
    #print("length of each layers:", length_of_each_layer)

    # 所有端到端的路
    all_path_side2side = []
    for path in all_path:
        if len(path) == 2:
            all_path_side2side.append(path)
        else:
            for i in range(len(path)-1):
                all_path_side2side.append([path[i],path[i+1]])

    # 定义节点到绘图的字典
    dict_nodes_graph = dict()
    count_number_layer = [0] * max_length
    for nodes in nodes_tuples_fixed_list_final:
        node, layer = nodes
        dict_nodes_graph[node] = [count_number_layer[layer], layer]
        count_number_layer[layer] += 1
    list_nodes_number.append(np.sum(length_of_each_layer))
    list_connection_number.append(len(all_path_side2side))
    #print("Number of nodes:{} Number of connections:{}".format(np.sum(length_of_each_layer),len(all_path_side2side)))
print("Total nodes:{} Total connections::{}".format(np.sum(list_nodes_number), np.sum(list_connection_number)))

Total nodes:135 Total connections::2154
